In [1]:
import sys
# sys.path.append('/Users/maxim_anisimov/Desktop/github_repos/tennis-matches')
from utils import get_atp_matches, get_atp_betting_odds

In [2]:
first_year = 2000
last_year = 2022
matches = get_atp_matches(first_year=first_year, last_year=last_year)
betting_odds = get_atp_betting_odds(first_year=first_year, last_year=last_year)

In [3]:
matches.head()

tourney_id tourney_name surface  draw_size  \
Tournament Start Date match_num                                              
2000-01-03            1           2000-339     Adelaide    Hard         32   
                      1           2000-451         Doha    Hard         32   
                      1           2000-891      Chennai    Hard         32   
                      2           2000-339     Adelaide    Hard         32   
                      2           2000-451         Doha    Hard         32   

                                tourney_level  tourney_date  winner_id  \
Tournament Start Date match_num                                          
2000-01-03            1                     A      20000103     102358   
                      1                     A      20000103     103017   
                      1                     A      20000103     103420   
                      2                     A      20000103     103819   
                      2                     A      20000103     101750   

                                winner_seed winner_entry        winner_name  \
Tournament Start Date match_num                                               
2000-01-03            1                   1          NaN     Thomas Enqvist   
                      1                   1          NaN     Nicolas Kiefer   
                      1                 NaN          NaN      Michal Tabara   
                      2                 NaN          NaN      Roger Federer   
                      2                 NaN            Q  Cristiano Caratti   

                                 ... l_1stIn  l_1stWon l_2ndWon  l_SvGms  \
Tournament Start Date match_num  ...                                       
2000-01-03            1          ...    37.0      25.0     13.0      9.0   
                      1          ...    53.0      36.0     17.0     12.0   
                      1          ...    23.0      16.0     13.0      9.0   
                      2          ...    15.0      13.0     12.0      8.0   
                      2          ...    57.0      41.0     18.0     16.0   

                                 l_bpSaved l_bpFaced winner_rank  \
Tournament Start Date match_num                                    
2000-01-03            1                2.0       4.0         4.0   
                      1                2.0       4.0         6.0   
                      1                3.0       7.0       123.0   
                      2                0.0       3.0        64.0   
                      2                2.0       7.0       213.0   

                                winner_rank_points loser_rank  \
Tournament Start Date match_num                                 
2000-01-03            1                     2606.0       56.0   
                      1                     2447.0       55.0   
                      1                      374.0        2.0   
                      2                      749.0       91.0   
                      2                      190.0      221.0   

                                 loser_rank_points  
Tournament Start Date match_num                     
2000-01-03            1                      805.0  
                      1                      814.0  
                      1                     3465.0  
                      2                      525.0  
                      2                      185.0  

[5 rows x 48 columns]

In [4]:
betting_odds.head()

,ATP,Location,Tournament,Series,Court,Surface,Round,Best of,Winner,Loser,...,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,1,Adelaide,Australian Hardcourt Championships,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,1,Adelaide,Australian Hardcourt Championships,International,Outdoor,Hard,1st Round,3,Enqvist T.,Clement A.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,1,Adelaide,Australian Hardcourt Championships,International,Outdoor,Hard,1st Round,3,Escude N.,Baccanello P.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,1,Adelaide,Australian Hardcourt Championships,International,Outdoor,Hard,1st Round,3,Federer R.,Knippschild J.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,1,Adelaide,Australian Hardcourt Championships,International,Outdoor,Hard,1st Round,3,Fromberg R.,Woodbridge T.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Prepare datasets for concatenation

In [9]:
betting_odds[betting_odds['Winner'].apply(lambda s: len(s.split())) == betting_odds['Winner'].apply(lambda s: len(s.split())).max()]

,ATP,Location,Tournament,Series,Court,Surface,Round,Best of,Winner,Loser,...,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
Date,,,,,,,,,,,,,,,,,,,,,
2016-07-18,44,Kitzbuhel,Generali Open,ATP250,Outdoor,Clay,1st Round,3,Munoz De La Nava D.,Berrer M.,...,NaN,NaN,1.57,2.25,NaN,NaN,1.83,2.25,1.72,2.08


#### Dictionary: from long to short names

In [21]:
betting_unique_names = sorted(list(set(betting_odds['Winner'].unique().tolist() + betting_odds['Loser'].unique().tolist())))

In [29]:
[name for name in betting_unique_names if '.' not in name]
# all others have abbreviations, so we will use the rule that <sth>. is a part of the name, all other experessions are a part of a surname.

['Aragone JC',
 'Hernandez-Fernandez J',
 'Kim K',
 'Meligeni Rodrigues F',
 'Zhang Ze']

In [36]:
betting_forenames = [''.join([elem for elem in name.split() if '.' in elem]) for name in betting_unique_names if '.' in name]
betting_surnames = [''.join([elem for elem in name.split() if '.' not in elem]) for name in betting_unique_names if '.' in name]

In [38]:
matches_unique_names = sorted(list(set(matches['winner_name'].unique().tolist() + matches['loser_name'].unique().tolist())))

In [44]:
from collections import defaultdict
default_dict = defaultdict(lambda: [])

In [47]:
betting_to_matches_names = {}
for forename, surname in zip(betting_forenames, betting_surnames):
    betting_to_matches_names[surname + ' ' + forename] = []
    for matches_full_name in matches_unique_names:
        if surname in matches_full_name:
            betting_to_matches_names[surname + ' ' + forename].append(matches_full_name)

In [48]:
betting_to_matches_names

{'Hajek J.': ['Jan Hajek'],
 'Abdulla M.': ['Abdulla Hajji', 'Abdullah Maqdas', 'Mohammed Abdulla'],
 'Abel M.': ['Maximilian Abel'],
 'Acasuso J.': ['Jose Acasuso'],
 'Adaktusson J.': ['Jacob Adaktusson'],
 'Agaev E.': ['Emin Agaev'],
 'Agamenone F.': ['Franco Agamenone'],
 'Agassi A.': ['Andre Agassi'],
 'Agenor R.': ['Ronald Agenor'],
 'Ager J.': ['Johannes Ager'],
 'Agostinelli B.': ['Bruno Agostinelli'],
 'Aguilar J.': ['Jorge Aguilar'],
 'Ahouda A.': ['Amine Ahouda'],
 'Ajdukovic D.': ['Duje Ajdukovic'],
 'AlGhareeb M.': [],
 'AlKhulaifi N.G.': [],
 'AlMutawa J.': [],
 'Al-Alawi S.K.': [],
 'Al-Ghareeb M.': [],
 'Alami K.': ['Karim Alami'],
 'Alawadhi O.': ['Abdulrahman Alawadhi'],
 'Albert M.': ['Albert Costa',
  'Albert Montanes',
  'Albert Portas',
  'Albert Ramos',
  'Alberto Berasategui',
  'Alberto Brause',
  'Alberto Brizzi',
  'Alberto Emmanuel Alvarado Larin',
  'Alberto Lim',
  'Alberto Martin',
  'Juan Albert Viloca Puig',
  'Mariano Albert Ferrando',
  'Mario Alberto 

In [53]:
import pandas
pandas.DataFrame(dict([ (k,pandas.Series(v)) for k,v in betting_to_matches_names.items() ])).T

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


,0,1,2,3,4,5,6,7,8,9,...,84,85,86,87,88,89,90,91,92,93
Hajek J.,Jan Hajek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Abdulla M.,Abdulla Hajji,Abdullah Maqdas,Mohammed Abdulla,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Abel M.,Maximilian Abel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Acasuso J.,Jose Acasuso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adaktusson J.,Jacob Adaktusson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
diPasquale A.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vanGemerden M.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vanLottum J.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vanScheppingen D.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
final_mapping = {}
for key in betting_to_matches_names:
    print(key)
    example = betting_to_matches_names[key].copy()
    remaining_stuff1 = {elem: [e for e in elem.split() if e not in key.split()] for elem in example}
    remaining_stuff2 = {elem: [e for e in key.split() if e not in elem.split()] for elem in example}

    remaining_stuff = {}
    for inner_key in remaining_stuff1:
        remaining_stuff[inner_key] = list(zip(remaining_stuff1[inner_key], remaining_stuff2[inner_key]))

    final_mapping[key] = []
    for full_name, rs in remaining_stuff.items():
        if all(inner_rs[0][0] + '.' == inner_rs[1] for inner_rs in rs):
            final_mapping[key].append(full_name)

    if  len(final_mapping[key]) == 0:
        print('No mapping has been found!')
    elif len(final_mapping[key]) == 1:
        print('Unique mapping has been found!')
    else:
        print('Multiple matches have been found!')

    print(100*'-')

final_mapping = {key: value[0] if len(value) == 1 else None for key, value in final_mapping.items()}
reversed_final_mapping = dict(zip(final_mapping.values(), final_mapping.keys()))

Hajek J.
Unique mapping has been found!
----------------------------------------------------------------------------------------------------
Abdulla M.
Unique mapping has been found!
----------------------------------------------------------------------------------------------------
Abel M.
Unique mapping has been found!
----------------------------------------------------------------------------------------------------
Acasuso J.
Unique mapping has been found!
----------------------------------------------------------------------------------------------------
Adaktusson J.
Unique mapping has been found!
----------------------------------------------------------------------------------------------------
Agaev E.
Unique mapping has been found!
----------------------------------------------------------------------------------------------------
Agamenone F.
Unique mapping has been found!
----------------------------------------------------------------------------------------------------
A

In [113]:
matches['Winner'] = [reversed_final_mapping.get(name) for name in matches['winner_name']]
matches['Loser'] = [reversed_final_mapping.get(name) for name in matches['loser_name']]

In [126]:
betting_odds['Year'] = betting_odds.index.year
matches['Year'] = matches.index.get_level_values(0).year

In [172]:
matches_odds = pandas.merge(matches.reset_index(), betting_odds.reset_index(), on=['Year', 'Winner', 'Loser'], how="outer")
matches_odds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91238 entries, 0 to 91237
Columns: 105 entries, Tournament Start Date to AvgL
dtypes: datetime64[ns](2), float64(70), int64(1), object(32)
memory usage: 73.8+ MB


In [173]:
matches_odds['date_difference_days'] = (matches_odds['Date'] - matches_odds['Tournament Start Date']).dt.days
matches_odds = matches_odds[matches_odds['date_difference_days'] >= 0] # TODO: how about qualifications?
matches_odds = matches_odds.loc[
    ((matches_odds['date_difference_days'] <= 14) & (matches_odds['Series'] == 'Grand Slam')) | (matches_odds['date_difference_days'] <= 7)
]
matches_odds.sort_values('Date', ascending=True, inplace=True, kind='quicksort', na_position='last')
matches_odds.set_index('Date', inplace=True)

In [174]:
matches_odds.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 46377 entries, 2000-01-03 to 2022-07-17
Columns: 105 entries, Tournament Start Date to date_difference_days
dtypes: datetime64[ns](1), float64(71), int64(1), object(32)
memory usage: 37.5+ MB


In [175]:
matches_odds

,Tournament Start Date,match_num,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,winner_id,winner_seed,...,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL,date_difference_days
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,2000-01-03,1.0,2000-339,Adelaide,Hard,32.0,A,20000103.0,102358.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2000-01-03,2000-01-03,23.0,2000-891,Chennai,Hard,32.0,A,20000103.0,102106.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2000-01-03,2000-01-03,23.0,2000-451,Doha,Hard,32.0,A,20000103.0,102783.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2000-01-03,2000-01-03,23.0,2000-339,Adelaide,Hard,32.0,A,20000103.0,102765.0,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2000-01-03,2000-01-03,22.0,2000-891,Chennai,Hard,32.0,A,20000103.0,102538.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-16,2022-07-11,298.0,2022-0315,Newport,Grass,32.0,A,20220711.0,202195.0,4,...,NaN,NaN,NaN,NaN,NaN,1.90,2.13,1.78,2.02,5.0
2022-07-16,2022-07-11,298.0,2022-0316,Bastad,Clay,32.0,A,20220711.0,202104.0,8,...,NaN,NaN,NaN,NaN,NaN,2.26,1.83,2.08,1.74,5.0
2022-07-16,2022-07-11,299.0,2022-0315,Newport,Grass,32.0,A,20220711.0,122330.0,3,...,NaN,NaN,NaN,NaN,NaN,1.62,2.59,1.56,2.40,5.0


In [177]:
# Odds stats have too many missing values
matches_odds[['MaxW', 'MaxL', 'AvgW', 'AvgL']].isna().mean()

MaxW    0.512646
MaxL    0.512646
AvgW    0.512646
AvgL    0.512646
dtype: float64

In [178]:
bookmakers = ['CB', 'GB', 'IW', 'SB', 'B365', 'B&W', 'EX', 'PS', 'UB', 'LB', 'SJ']

In [179]:
matches_odds['MaxW'] = matches_odds[[bm + 'W' for bm in bookmakers]].max(axis=1)
matches_odds['MaxL'] = matches_odds[[bm + 'L' for bm in bookmakers]].max(axis=1)

matches_odds['AvgW'] = matches_odds[[bm + 'W' for bm in bookmakers]].mean(axis=1)
matches_odds['AvgL'] = matches_odds[[bm + 'L' for bm in bookmakers]].mean(axis=1)

In [180]:
betting_odds_columns = [bm + 'W' for bm in bookmakers] + [bm + 'L' for bm in bookmakers]

In [181]:
matches_odds[['MaxW', 'MaxL', 'AvgW', 'AvgL']].isna().mean()

MaxW    0.070401
MaxL    0.070380
AvgW    0.070401
AvgL    0.070380
dtype: float64

In [182]:
matches_odds['AvgW'].isnull().sum() / len(matches_odds)

0.07040127649481424

In [183]:
matches_odds_nona_avg_odds = matches_odds.dropna(subset=['AvgW', 'AvgL'])

In [187]:
matches_odds_nona_avg_odds['bookmaking_market_right'] = matches_odds_nona_avg_odds['AvgW'] < matches_odds_nona_avg_odds['AvgL']
matches_odds_nona_avg_odds['ranking_right'] = matches_odds_nona_avg_odds['winner_rank'] < matches_odds_nona_avg_odds['loser_rank'] 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [189]:
matches_odds_nona_avg_odds['bookmaking_market_right'].mean()

0.6963026535535349

In [190]:
matches_odds_nona_avg_odds['ranking_right'].mean()

0.6516283169419187

In [191]:
import plotly.express as px

In [192]:
bookmaking_market_right_per_year = matches_odds_nona_avg_odds['bookmaking_market_right'].groupby(matches_odds_nona_avg_odds.index.year).mean()
ranking_right_per_year = matches_odds_nona_avg_odds['ranking_right'].groupby(matches_odds_nona_avg_odds.index.year).mean()

In [197]:
matches_odds[['surface', 'Surface']].isna().mean()

surface    0.0
Surface    0.0
dtype: float64

In [201]:
bookmaking_market_right_per_surface = matches_odds_nona_avg_odds.groupby('Surface')['bookmaking_market_right'].mean()
ranking_right_per_surface = matches_odds_nona_avg_odds.groupby('Surface')['ranking_right'].mean()

In [195]:
px.line(
    pandas.DataFrame({'Bookmaking market': bookmaking_market_right_per_year, 'Ranking': ranking_right_per_year})
)

In [211]:
guessing_per_surface_to_plot = pandas.concat(
    [
        pandas.DataFrame(
        {'Type': 'Bookmaking Market', 'Surface':bookmaking_market_right_per_surface.index, 'Right prop': bookmaking_market_right_per_surface.values}
    ),
    pandas.DataFrame(
        {'Type': 'Ranking', 'Surface':ranking_right_per_surface.index, 'Right prop': ranking_right_per_surface.values}
    )
])

In [212]:
guessing_per_surface_to_plot

,Type,Surface,Right prop
0,Bookmaking Market,Carpet,0.667164
1,Bookmaking Market,Clay,0.692838
2,Bookmaking Market,Grass,0.713351
3,Bookmaking Market,Hard,0.696180
0,Ranking,Carpet,0.611194
1,Ranking,Clay,0.645092
2,Ranking,Grass,0.652174
3,Ranking,Hard,0.657671


In [213]:
px.bar(guessing_per_surface_to_plot, x='Surface', y='Right prop', color='Type', barmode='group')

In [222]:
bookmaking_market_right_per_year_and_surface = matches_odds_nona_avg_odds.groupby([matches_odds_nona_avg_odds.index.year, 'Surface'])['bookmaking_market_right'].mean()
ranking_right_per_year_and_surface = matches_odds_nona_avg_odds.groupby([matches_odds_nona_avg_odds.index.year, 'Surface'])['ranking_right'].mean()

In [223]:
bookmaking_market_right_per_year_and_surface.reset_index()

,Date,Surface,bookmaking_market_right
0,2001,Carpet,0.620000
1,2001,Clay,0.646220
2,2001,Grass,0.692308
3,2001,Hard,0.691763
4,2002,Carpet,0.721805
...,...,...,...
69,2021,Grass,0.717489
70,2021,Hard,0.677820
71,2022,Clay,0.704805
72,2022,Grass,0.714876


In [224]:
px.line(
    bookmaking_market_right_per_year_and_surface.reset_index(),
     x='Date', y='bookmaking_market_right', color='Surface',
)

In [226]:
px.line(
    ranking_right_per_year_and_surface.reset_index(),
     x='Date', y='ranking_right', color='Surface',
)

### Model data
Where there are columns player 1 and player 2